<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reading-csv-with-customSchema" data-toc-modified-id="Reading-csv-with-customSchema-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reading csv with customSchema</a></span></li><li><span><a href="#Type-inference-by-Reflection" data-toc-modified-id="Type-inference-by-Reflection-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Type inference by Reflection</a></span></li></ul></div>

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder.
    appName("test").
    enableHiveSupport().
    config("spark.sql.warehouse.dir","/user/hive/warehouse").
    getOrCreate()
)

spark.sparkContext.getConf().getAll()

[('spark.driver.port', '10879'),
 ('spark.app.id', 'local-1638900019272'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.199.77'),
 ('spark.hadoop.fs.default.name', 'hdfs://localhost:9000'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.sql.warehouse.dir', '/user/hive/warehouse'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1638900018523'),
 ('spark.hadoop.fs.defaultFS', 'hdfs://localhost:9000'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

# Reading csv with customSchema

In [4]:
from pyspark.sql.types import *

customSchema = StructType([
    StructField("id", IntegerType(), True),
    StructField("bikes_available", IntegerType(), True),
    StructField("docks_available", IntegerType(), True),
    StructField("time", TimestampType(), True)
])

path = "file:///home/sergey/Py_SparkDataFrame_edx_CS105_CS110_CS120/data/201408_status_data.csv"
df = sqlContext.read.csv(path, header=True, schema=customSchema)
df.show(5)

+---+---------------+---------------+--------------------+
| id|bikes_available|docks_available|                time|
+---+---------------+---------------+--------------------+
|  2|             12|             15|2014-03-01 00:00:...|
|  2|             12|             15|2014-03-01 00:01:...|
|  2|             12|             15|2014-03-01 00:02:...|
|  2|             12|             15|2014-03-01 00:03:...|
|  2|             12|             15|2014-03-01 00:04:...|
+---+---------------+---------------+--------------------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- bikes_available: integer (nullable = true)
 |-- docks_available: integer (nullable = true)
 |-- time: timestamp (nullable = true)



# Type inference by Reflection

In [31]:
from pyspark.sql import Row
from datetime import datetime as dt
rdd = sc.textFile(path).zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])

rowRDD = (rdd
          .map(lambda x: x.split(","))
          .map(lambda x: Row(id_ = int(x[0][1:-1])
                             ,bikes_available = int(x[1][1:-1])
                             ,docks_available = int(x[2][1:-1])
                             ,time = dt.strptime(x[3][1:-1], "%Y-%m-%d %H:%M:%S")
                            )
              )
         )

df2 = rowRDD.toDF()
df2.show(5)

+---------------+---------------+---+--------------------+
|bikes_available|docks_available|id_|                time|
+---------------+---------------+---+--------------------+
|             12|             15|  2|2014-03-01 00:00:...|
|             12|             15|  2|2014-03-01 00:01:...|
|             12|             15|  2|2014-03-01 00:02:...|
|             12|             15|  2|2014-03-01 00:03:...|
|             12|             15|  2|2014-03-01 00:04:...|
+---------------+---------------+---+--------------------+
only showing top 5 rows



In [32]:
df2.printSchema()

root
 |-- bikes_available: long (nullable = true)
 |-- docks_available: long (nullable = true)
 |-- id_: long (nullable = true)
 |-- time: timestamp (nullable = true)

